# Reproducibility - resume

1. Create and commit two versions of dataframe -> v1, v2
2. Run flow 1 with v1 data -> crashes b/c flow 2 hasn't run
3. Run flow 2 with v2 data
4. Resume flow 1 successfully

In [26]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

from doltpy.core import Dolt
from doltpy.core.write import import_df
import pandas as pd

dolt = Dolt.init("foo")

df_v1 = pd.DataFrame({"A": [1,1,1], "B": [1,1,1]})
df_v2 = pd.DataFrame({"A": [1,1,1,2,2,2], "B": [1,1,1,2,2,2]})

import_df(dolt, "bar", df_v1.reset_index(), ["index"], "create")
dolt.add("bar")
dolt.commit("Initialize bar")

v1 = list(dolt.log(number="1").keys())[0]

import_df(dolt, "bar", df_v2.reset_index(), ["index"], "update")
dolt.add("bar")
dolt.commit("Add rows to bar")

v2 = list(dolt.log(number="1").keys())[0]

In [24]:
!cat succeeds_second.py

import datetime
import pickle

from metaflow import FlowSpec, step, DoltDT, Parameter, Flow
import pandas as pd
from sklearn import tree

class SucceedsSecondDemo(FlowSpec):

    bar_version = Parameter('bar-version',  help="Specifc the tag for the input version", required=True)

    @step
    def start(self):
        with DoltDT(run=self, database='foo', branch="master") as dolt:
            self.df = dolt.read_table('bar')

        first_run = Flow("SucceedsFirstDemo").latest_successful_run
        first_run_ts = datetime.datetime.strptime(first_run.finished_at, "%Y-%m-%dT%H:%M:%SZ")
        if first_run_ts < (datetime.datetime.now() - datetime.timedelta(minutes=1)):
            raise Exception("Run `FirstDemo` within one minute of `SecondDemo`")

        self.next(self.middle)

    @step
    def middle(self):
        with DoltDT(run=self, database='foo', branch="master") as dolt:
            df = self.df
            df["B"] = df["B"].map(lambda x: x*2)

            dolt.write_table(

In [28]:
!cat succeeds_first.py

import pickle

from metaflow import FlowSpec, step, DoltDT, Parameter
import pandas as pd
from sklearn import tree

class SucceedsFirstDemo(FlowSpec):

    bar_version = Parameter('bar-version',  help="Specifc the tag for the input version", required=True)

    @step
    def start(self):
        with DoltDT(run=self, database='foo', branch="master") as dolt:
            self.df = dolt.read_table('bar', commit=self.bar_version)

        self.next(self.middle)

    @step
    def middle(self):
        with DoltDT(run=self, database='foo', branch="master") as dolt:

            df = self.df
            df["B"] = df["B"].map(lambda x: x*2)

            dolt.write_table(table_name='baz', df=df, pks=['index'])

        self.next(self.end)

    @step
    def end(self):
        pass


if __name__ == '__main__':
    SucceedsFirstDemo()


In [44]:
!poetry run python3 succeeds_second.py run --bar-version $v1

Metaflow 2.2.5.post23+git20868e9 executing SucceedsSecondDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-17 18:12:26.846 Workflow starting (run-id 1610935946839873):
2021-01-17 18:12:26.857 [1610935946839873/start/1 (pid 30394)] Task is starting.
2021-01-17 18:12:28.231 [1610935946839873/start/1 (pid 30394)] 01-17 18:12:28 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 18:12:28.291 [1610935946839873/start/1 (pid 30394)] 01-17 18:12:28 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 18:12:28.546 [1610935946839873/start/1 (pid 30394)] 01-17 18:12:28 doltpy.core.dolt INFO     * master                                        	bp7f940q1914r898fcntcnhhaihg8c7t
2021-01-17 18:12:28.735 [1610935946839873/start/1 (pid 30394)] 
2021-01-17 18:12:28.735 [1610935946839873/start/1 (pid 30394)] 01-17 18:12:28 

In [34]:
!poetry run python3 succeeds_first.py run --bar-version $v2

Metaflow 2.2.5.post23+git20868e9 executing SucceedsFirstDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-17 18:04:46.198 Workflow starting (run-id 1610935486185489):
2021-01-17 18:04:46.206 [1610935486185489/start/1 (pid 29757)] Task is starting.
2021-01-17 18:04:47.423 [1610935486185489/start/1 (pid 29757)] 01-17 18:04:47 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 18:04:47.455 [1610935486185489/start/1 (pid 29757)] 01-17 18:04:47 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 18:04:47.514 [1610935486185489/start/1 (pid 29757)] 01-17 18:04:47 doltpy.core.dolt INFO     * master                                        	6i8nk68bj3rq8rv1b63e7tu309utv451
2021-01-17 18:04:47.680 [1610935486185489/start/1 (pid 29757)] 
2021-01-17 18:04:47.680 [1610935486185489/start/1 (pid 29757)] 01-17 18:04:47 d

2021-01-17 18:04:50.550 [1610935486185489/middle/2 (pid 29791)] Task finished successfully.
2021-01-17 18:04:50.557 [1610935486185489/end/3 (pid 29831)] Task is starting.
2021-01-17 18:04:51.743 [1610935486185489/end/3 (pid 29831)] 01-17 18:04:51 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
2021-01-17 18:04:51.750 [1610935486185489/end/3 (pid 29831)] 01-17 18:04:51 doltpy.core.system_helpers INFO     No processes to clean up, exiting
2021-01-17 18:04:51.900 [1610935486185489/end/3 (pid 29831)] Task finished successfully.
2021-01-17 18:04:51.901 Done!
01-17 18:04:51 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
01-17 18:04:51 doltpy.core.system_helpers INFO     No processes to clean up, exiting


In [41]:
!poetry run python3 succeeds_second.py resume start

Metaflow 2.2.5.post23+git20868e9 executing SucceedsSecondDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-17 18:10:54.184 Gathering required information to resume run (this may take a bit of time)...
2021-01-17 18:10:54.191 Workflow starting (run-id 1610935854183920):
2021-01-17 18:10:54.199 [1610935854183920/start/1 (pid 30142)] Task is starting.
2021-01-17 18:10:55.292 [1610935854183920/start/1 (pid 30142)] 01-17 18:10:55 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 18:10:55.319 [1610935854183920/start/1 (pid 30142)] 01-17 18:10:55 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 18:10:55.380 [1610935854183920/start/1 (pid 30142)] 01-17 18:10:55 doltpy.core.dolt INFO     * master                                        	bp7f940q1914r898fcntcnhhaihg8c7t
2021-01-17 18:10:55.543 [161093585418392